# Task 3:
Multiple Mixture components using labeled data: You will relax the assumption made in the
first 2 experiments. You will consider that a single news article can belong to several subtopics
and experiment with a Naive Bayes classifier using multiple mixture components on the labeled
dataset.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import metrics
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
train = pd.read_csv('../data/train_cleaned_stemed_reuters.csv', delimiter = ',')
test = pd.read_csv('../data/test_cleaned_stemed_reuters.csv', delimiter = ',')
train 

Unnamed: 0                                                  X  \
0              0  bahia cocoa review shower continu throughout w...   
1              1  comput termin system lt cpml complet sale comp...   
2              2  n z trade bank deposit growth rise slightli ne...   
3              3  nation amus up viacom lt via bid viacom intern...   
4              4  roger lt rog see st qtr net significantli roge...   
...          ...                                                ...   
7764        7764  u k money market shortag forecast revis bank e...   
7765        7765  knight ridder inc lt krn set quarterli qtli di...   
7766        7766  technitrol inc lt tnl set quarterli qtli div c...   
7767        7767  nationwid cellular servic inc lt ncel th qtr s...   
7768        7768  lt h automot technolog corp year net shr ct vs...   

                             Y  
0                    ['cocoa']  
1                      ['acq']  
2             ['money-supply']  
3                      ['acq']  
4                     ['earn']  
...                        ...  
7764  ['interest', 'money-fx']  
7765                  ['earn']  
7766                  ['earn']  
7767                  ['earn']  
7768                  ['earn']  

[7769 rows x 3 columns]

In [3]:
train['Y'] = train.Y.apply(lambda x: re.sub('\s+', '', x))
train['Y'] = train.Y.apply(lambda x: re.sub('\'', '', x))
train['Y'] = train.Y.apply(lambda x: x[1:-1].split(','))

train["Y"][7764]

['interest', 'money-fx']

In [4]:
test['Y'] = test.Y.apply(lambda x: re.sub('\s+', '', x))
test['Y'] = test.Y.apply(lambda x: re.sub('\'', '', x))
test['Y'] = test.Y.apply(lambda x: x[1:-1].split(','))

In [5]:
train_X = np.array(train["X"])
train_Y = np.array(train["Y"])
train_X.shape

(7769,)

In [6]:
test_X = np.array(test["X"])
test_Y = np.array(test["Y"])
test_X[:2]

array(['asian export fear damag u japan rift mount trade friction u japan rais fear among mani asia export nation row could inflict far reach econom damag businessmen offici said told reuter correspond asian capit u move japan might boost protectionist sentiment u lead curb american import product export said conflict would hurt long run short term tokyo loss might gain u said impos mln dlr tariff import japanes electron good april retali japan alleg failur stick pact sell semiconductor world market cost unoffici japanes estim put impact tariff billion dlr spokesmen major electron firm said would virtual halt export product hit new tax abl busi said spokesman lead japanes electron firm matsushita electr industri co ltd lt mc tariff remain place length time beyond month mean complet eros export good subject tariff u said tom murtha stock analyst tokyo offic broker lt jame capel co taiwan businessmen offici also worri awar serious u threat japan serv warn us said senior taiwanes trade of

In [7]:
vectorizer = TfidfVectorizer()
train_data_vectors = vectorizer.fit_transform(train_X)
train_data_vectors.shape

(7769, 18034)

In [8]:
test_data_vectors = vectorizer.transform(test_X)
test_data_vectors.shape

(3019, 18034)

In [9]:
bayes_classifier = MultinomialNB(alpha=0.01)
bayes_classifier

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
new_y = mlb.fit_transform(train_Y)
new_y

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
new_y.shape

(7769, 90)

In [12]:
for n in new_y:
    if sum(n) > 1:
        print(n)
        break

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]


In [13]:
# bayes_classifier.fit(train_data_vectors,new_y)

from sklearn.multioutput import MultiOutputClassifier
multi_target = MultiOutputClassifier(bayes_classifier, n_jobs=-1)
multi_target.fit(train_data_vectors,new_y)


MultiOutputClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None,
                                              fit_prior=True),
                      n_jobs=-1)

In [14]:
test_predict = multi_target.predict(test_data_vectors)
test_predict

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
test_predict[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0])

In [16]:
new_y_test = mlb.transform(test_Y)
new_y_test[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0])

In [18]:
acc = accuracy_score(new_y_test, test_predict)
acc

0.6840013249420338

In [ ]:
test_Y

In [ ]:
type(test_predict[0])